# **Projeto 2 introdução a redes neurais - Modelo não supervisionado**

--------------------

## Objetivo: 

Implementar uma rede neural não supervisionada (e.g., SOM ou GNG) em pelo menos dois datasets, avaliar os padrões detectados em cada conjunto como custers e outliers. Avaliar a qualidade dos agrupamentos variando parâmetros como número de neurônios, taxas de aprendizado e grid.

## Importando bibliotecas necessárias

In [6]:
import numpy as np
import seaborn as sns
import cv2
import os 

# Configuração do Searborn
sns.set(
    style='darkgrid',
    context='notebook',
    rc={
        'axes.spines.top': False,
        'axes.spines.right': False
    }
)

## Primeiro Dataset

O primeiro dataset é o Chest X-ray images (Pneumonia), é um dataset do *kaggle* (`https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia`) que contém 5856 imagens de raios-X de tórax, dividas em duas classes: pneumonia (bacteriana e viral) e sem pneumonia (normal). As imagens foram rotuladas manualmente.  

## Preparação dos dados

### As seguintes etapas serão realizadas: 
 - Leitura das imagens: As imagens são divididas em pastas com as classes "PNEUMONIA" e "NORMAL".
 - Redimensionamento: Cada imagem é redimensionada para um tamanho fixo.
 - Transformação: As imagens são achatadas em vetores para o processamento da rede.
 - Codificação de rótulos: As classes são convertidas para valores binários.
 - Normalização dos dados: O conjunto de dados é escalado para garantir que todas as variáveis estejam na mesma escala.

In [7]:
# Caminhos para os dados de treino / teste 

diretorio_treino = "./train"
diretorio_teste = "./test"

A biblioteca `os` será utilizada para navegar pelas pastas e acessar os arquivos. A função `os.listdir()` lista os arquivos dentro de um diretório especificado, enquanto `os.path.join()` é usada para construir caminhos de arquivos de maneira compatível com diferentes sistemas operacionais, garantindo a integridade do código em diferentes plataformas, já para a preparação das imagens a biblioteca OpenCV (cv2) será usada para carregar e pré-processar as imagens. A função `cv2.imread()` lê a imagem e a converte para escala cinza, enquanto `cv2.resize()` redimensiona todas as imagens para um tamanho uniforme. Em seguida, a imagem é achatada com o `img.flatten()` para transforma-lá em um vetor

In [8]:
# Função para carregar e processar as imagens do dataset

def carregar_imagens(diretorio, tamanho=(64, 64)): 
    """ 
    Função para carregar e preprocessar as imagens do diretório. 
    
    Parâmetros: 
    - diretorio: Caminho para o diretório contendo as imagens.
    - tamanho: Tamanho para o qual as imagens serão redimensionadas (64x64).
    
    Retorna: 
    - imagens: Array numpy com as imagens transformadas em vetores.
    - rotulos_imagens: Array numpy com os rótulos (0 para "NORMAL" e 1 para "PNEUMONIA").
    """

    # Listas para armazenar as imagens e seus respectivos rótulos
    imagens = []
    rotulos_imagens = []

    # Loop para percorrer as pastas de imagens ("PNEUMONIA" e "NORMAL") dentro do diretório

    for rotulo in os.listdir(diretorio): 
        pasta_rotulo = os.path.join(diretorio, rotulo)  # Caminho completo da pasta de rótulo
        # Loop para percorrer os arquivos dentro da pasta de rótulo
        for arquivo in os.listdir(pasta_rotulo): 
            caminho = os.path.join(pasta_rotulo, arquivo)  # Caminho completo da imagem
            # Ler a imagem em escala cinza
            imagem = cv2.imread(caminho, cv2.IMREAD_GRAYSCALE)
            # Redimensionar a imagem
            imagem = cv2.resize(imagem, tamanho)
            # Achatar a imagem para um vetor
            imagens.append(imagem.flatten())
            # Codificação dos rótulos, 0 : NORMAL, 1 : PNEUMONIA
            rotulos_imagens.append(0 if rotulo == "NORMAL" else 1)

    # Retorna as imagens e os rótulos como arrays numpy
    return np.array(imagens), np.array(rotulos_imagens) 

# Carregar as imagens de treino e teste usando a função 'carregar_imagens'
dados_treino, labels_treino = carregar_imagens(diretorio_treino)
dados_teste, labels_teste = carregar_imagens(diretorio_teste)

A normalização dos dados será realizada utilizando o `MinMaxScaler`, que escala os valores para um intervalo específico. A fórmula utilizada para isso é: 

$$
X_{\text{escalado}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}
$$

Em que: 

- $X$ é o valor original do dado.
- $X_{\text{min}}$ e $X_{\text{max}}$ são, respectivamente, o menor e o maior valor da feature.
- $X_{\text{escalado}}$ é o valor normalizado.

In [10]:
from sklearn.preprocessing import MinMaxScaler # Importando o MinMaxScaler
# Criando uma instância do MinMaxScaler
scaler = MinMaxScaler()
dados_treino_normalizados = scaler.fit_transform(dados_treino) # Normalizar os dados de treino (escala entre 0 e 1)
dados_teste_normalizados = scaler.transform(dados_teste) # Normalizar os dados de teste

## Modelo neural não supervisionado self-organizing maps (SOM)

`https://github.com/JustGlowing/minisom`